# Gated recurrent unit

Gated recurrent units (GRUs) are a gating mechanism in recurrent neural networks, introduced in 2014 by Kyunghyun Cho et al.
The GRU is like a long short-term memory (LSTM) with a forget gate, but has fewer parameters than LSTM, as it lacks an output gate. 

However, as shown by Gail Weiss, Yoav Goldberg and Eran Yahav, the LSTM is "strictly stronger" than the GRU as it can easily perform unbounded counting, while the GRU cannot. That's why the GRU fails to learn simple languages that are learnable by the LSTM.

Similarly, as shown by Denny Britz, Anna Goldie, Minh-Thang Luong and Quoc Le of Google Brain, LSTM cells consistently outperform GRU cells in "the first large-scale analysis of architecture variations for Neural Machine Translation."

In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F  

from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms

Let's define our hyperparameters that will be used:

In [2]:
input_size = 28
hidden_size = 256
num_layers = 2
num_classes = 10
sequence_length = 28
learning_rate = 0.001
batch_size = 64
num_epochs = 3

Load data:

In [3]:
train_dataset = datasets.MNIST('', train=True,
                               transform=transforms.ToTensor(), 
                               download=True)


test_dataset = datasets.MNIST('', train=False,
                               transform=transforms.ToTensor(), 
                               download=True)


train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [4]:
class RNN_GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out



Set the device:

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Initialize network:

In [6]:
model = RNN_GRU(input_size, hidden_size, num_layers, num_classes).to(device)

Let's calculate loss and specify the optimizer:

In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [8]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = loss_function(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    print(loss)

tensor(0.0142, grad_fn=<NllLossBackward>)
tensor(0.0483, grad_fn=<NllLossBackward>)
tensor(0.0034, grad_fn=<NllLossBackward>)


Let's check accuracy on training & test to see how good our model is:

In [9]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Accuracy on training data:")
    else:
        print("Accuracy on test data:")

    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )
    # Set model back to train
    model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Accuracy on training data:
Got 59123 / 60000 with accuracy 98.54
Accuracy on test data:
Got 9839 / 10000 with accuracy 98.39
